In [78]:
# Import dependencies
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import requests
import pymongo
import os
import time
from config import nasa_url, facts_url, mars_hemisphere_url

In [68]:
# URL of page to be scraped
url = nasa_url

# Retrieve page with the requests module
response = requests.get(url, verify=True)

# Parse HTMl
soup = BeautifulSoup(response.text, 'html.parser')

In [69]:
# Grab latest headline and tagline
headline = soup.find_all('div', class_="content_title")[0].text
print(headline)

# Grab the tagline for the above headline
tagline = soup.find_all('div', class_="rollover_description_inner")[0].text
print(tagline)



NASA's Perseverance Rover Collects Puzzle Pieces of Mars' History



The rocks it has analyzed for sample collection are helping the team better understand a past marked by volcanic activity and water.



In [85]:
# Use Splinter to visit site for featured image - First init Splinter
executable_path = {'executable_path': ChromeDriverManager().install()} # Ensure latest version is installed
browser = Browser('chrome', **executable_path, headless=False) # Initialize browser

# Visit URL
ft_img_grab_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(ft_img_grab_url)

# Open featured image full res destination
browser.links.find_by_partial_text('FULL IMAGE').click()
time.sleep(6)

# Parse scraped HTML
scraped_html = browser.html
img_parse = bs(scraped_html, 'html.parser')

# Grab full res image
featured_image_scrape = img_parse.find('img', class_="fancybox-image")
featured_image_url = featured_images.attrs['src']

# Get full link
featured_img_final = f"https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{featured_image_url}"
featured_img_final



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/josephchancey/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [94]:
# Closer Splinter browser
browser.quit()